# Análisis de Escritos según Ámbito 1: Aspectos de Estructura Global

El prompt, formulado usando CHATGPT, permite evaluar un texto de manera automática según los tres criterios de la rúbrica académica:  

1. **Estructura global: Introducción**  
   Evalúa si el texto presenta una introducción clara, bien contextualizada o simplemente mencionada.  

2. **Estructura global: Conclusión o cierre**  
   Revisa si el texto cierra correctamente con una conclusión, una proyección de la tesis o solo un comentario.  

3. **Progresión textual**  
   Analiza el uso de párrafos y si cada uno aporta una idea principal distinta.  

El resultado de la evaluación se entrega en **formato JSON** para facilitar su interpretación y posterior uso en sistemas automáticos o reportes.  

In [ ]:
import os
import json
import pandas as pd
from dotenv import load_dotenv
from openai import AzureOpenAI

load_dotenv()
API_KEY = os.getenv("AZURE_OPENAI_KEY")
ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
DEPLOYMENT_GPT = os.getenv("AZURE_OPENAI_DEPLOYMENT_GPT")
API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION")

client = AzureOpenAI(
    api_key=API_KEY,
    api_version=API_VERSION,
    azure_endpoint=ENDPOINT
)

data_path = os.path.join("..", "data", "escritos_seleccionados_a1.xlsx")
df = pd.read_excel(data_path)
textos = df["escrito"].dropna().tolist()

prompt_estructura = """
Evalúa el siguiente texto usando esta rúbrica:

1. Estructura global: Introducción
- Nivel 4: Se identifica claramente una introducción en la que se presenta y contextualiza el tema del texto (qué, quiénes, dónde, cuándo, etc.).
- Nivel 3: Se identifica claramente una introducción en la que se menciona el tema del texto, pero sin suficiente contextualización.
- Nivel 2: La estructura del texto no es clara, pero es posible inferir la presencia de una introducción amalgamada en otras oraciones o párrafos.
- Nivel 1: No es posible identificar una introducción en el texto.

2. Estructura global: Conclusión o cierre
- Nivel 4: En la estructura del texto se distingue un cierre en el que se concluye o se proyecta la tesis.
- Nivel 3: En la estructura del texto se distingue un cierre a modo de comentario.
- Nivel 2: La estructura del texto no es clara, pero es posible inferir la presencia de una conclusión o cierre amalgamada en otras oraciones o párrafos.
- Nivel 1: No se identifica una conclusión o cierre en el texto.

3. Progresión textual (uso de párrafos e inclusión de información principal)
- Nivel 4: El texto presenta párrafos claramente identificables y en cada uno se reconoce una información principal distinta a la del párrafo anterior.
- Nivel 3: El texto presenta párrafos claramente identificables, pero en un párrafo se repite información de algún párrafo anterior.
- Nivel 2: El texto presenta párrafos claramente identificables, pero en dos párrafos se repite información de algún párrafo anterior.
- Nivel 1: No se presentan párrafos claramente identificables o en todos los párrafos se repite información anterior.

Devuelve la evaluación **únicamente en formato JSON** con la siguiente estructura:

{{
  "introduccion": {{
    "nivel": (número de 1 a 4),
    "justificacion": "explicación breve"
  }},
  "conclusion_cierre": {{
    "nivel": (número de 1 a 4),
    "justificacion": "explicación breve"
  }},
  "progresion_textual": {{
    "nivel": (número de 1 a 4),
    "justificacion": "explicación breve"
  }}
}}

### Texto a evaluar:
{essay}

### INSTRUCCIONES CRÍTICAS:
- NO uses markdown, NO incluyas ```json, devuelve SOLO el JSON.
"""

resultados = []

for i, texto in enumerate(textos, start=1):    
    prompt_estructura_a1 = prompt_estructura.format(essay=texto)

    print(f"Procesando texto {i}/{len(textos)}...")

    messages = [
        {"role": "system", "content": 'Realiza las tareas de corrección que te solicite para un texto entregado'},
        {"role": "user", "content": prompt_estructura_a1}
    ]

    try:
        response = client.chat.completions.create(
            model=DEPLOYMENT_GPT,
            messages=messages,
            # temperature=0, (No es soportado en Azure OpenAI)
            response_format={"type": "json_object"}
        )

        respuesta_limpia = response.choices[0].message.content.strip()
        resultado_json = json.loads(respuesta_limpia)
        resultados.append({
            "id_texto": i,
            "resultado": resultado_json
        })

    except json.JSONDecodeError as e:
        print(f"Error al parsear JSON en texto {i}: {e}")
        print("Respuesta cruda recibida:")
        print(respuesta_limpia)

output_dir = os.path.join("..", "resultados", "06")
os.makedirs(output_dir, exist_ok=True)
output_path = os.path.join(output_dir, "a1_estructura.json")

with open(output_path, "w", encoding="utf-8") as f:
    json.dump(resultados, f, ensure_ascii=False, indent=4)

print(f"\nTodos los resultados guardados en {output_path}")

Procesando texto 1/20...
Procesando texto 2/20...
Procesando texto 3/20...
Procesando texto 4/20...
Procesando texto 5/20...
Procesando texto 6/20...
Procesando texto 7/20...
Procesando texto 8/20...
Procesando texto 9/20...
Procesando texto 10/20...
Procesando texto 11/20...
Procesando texto 12/20...
Procesando texto 13/20...
Procesando texto 14/20...
Procesando texto 15/20...
Procesando texto 16/20...
Procesando texto 17/20...
Procesando texto 18/20...
Procesando texto 19/20...
Procesando texto 20/20...

Todos los resultados guardados en ..\resultados\06\a1_estructura.json


In [4]:
import os
import json
import pandas as pd
from dotenv import load_dotenv
from openai import AzureOpenAI

load_dotenv()
API_KEY = os.getenv("AZURE_OPENAI_KEY")
ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
DEPLOYMENT_GPT = os.getenv("AZURE_OPENAI_DEPLOYMENT_GPT")
API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION")

client = AzureOpenAI(
    api_key=API_KEY,
    api_version=API_VERSION,
    azure_endpoint=ENDPOINT
)

data_path = os.path.join("..", "data", "escritos_seleccionados_a1.xlsx")
df = pd.read_excel(data_path)
textos = df["escrito"].dropna().tolist()

prompt_introduccion = """
Evalúa el siguiente texto usando esta rúbrica:

**Estructura global: Introducción**
- Nivel 4: Se identifica claramente una introducción en la que se presenta y contextualiza el tema del texto (qué, quiénes, dónde, cuándo, etc.).
- Nivel 3: Se identifica claramente una introducción en la que se menciona el tema del texto, pero sin suficiente contextualización.
- Nivel 2: La estructura del texto no es clara, pero es posible inferir la presencia de una introducción amalgamada en otras oraciones o párrafos.
- Nivel 1: No es posible identificar una introducción en el texto.

Devuelve la evaluación **únicamente en formato JSON** con la siguiente estructura:

{{
  "introduccion": {{
    "nivel": (número de 1 a 4),
    "justificacion": "explicación breve"
}}

### Texto a evaluar:
{essay}

### INSTRUCCIONES CRÍTICAS:
- NO uses markdown, NO incluyas ```json, devuelve SOLO el JSON.
"""

resultados = []

for i, texto in enumerate(textos, start=1):    
    prompt_introduccion_a1 = prompt_introduccion.format(essay=texto)

    print(f"Procesando texto {i}/{len(textos)}...")

    messages = [
        {"role": "system", "content": 'Realiza las tareas de corrección que te solicite para un texto entregado'},
        {"role": "user", "content": prompt_introduccion_a1}
    ]

    try:
        response = client.chat.completions.create(
            model=DEPLOYMENT_GPT,
            messages=messages,
            # temperature=0, (No es soportado en Azure OpenAI)
            response_format={"type": "json_object"}
        )

        respuesta_limpia = response.choices[0].message.content.strip()
        resultado_json = json.loads(respuesta_limpia)
        resultados.append({
            "id_texto": i,
            "resultado": resultado_json
        })

    except json.JSONDecodeError as e:
        print(f"Error al parsear JSON en texto {i}: {e}")
        print("Respuesta cruda recibida:")
        print(respuesta_limpia)

output_dir = os.path.join("..", "resultados", "06")
os.makedirs(output_dir, exist_ok=True)
output_path = os.path.join(output_dir, "a1_introduccion.json")

with open(output_path, "w", encoding="utf-8") as f:
    json.dump(resultados, f, ensure_ascii=False, indent=4)

print(f"\nTodos los resultados guardados en {output_path}")

Procesando texto 1/20...
Procesando texto 2/20...
Procesando texto 3/20...
Procesando texto 4/20...
Procesando texto 5/20...
Procesando texto 6/20...
Procesando texto 7/20...
Procesando texto 8/20...
Procesando texto 9/20...
Procesando texto 10/20...
Procesando texto 11/20...
Procesando texto 12/20...
Procesando texto 13/20...
Procesando texto 14/20...
Procesando texto 15/20...
Procesando texto 16/20...
Procesando texto 17/20...
Procesando texto 18/20...
Procesando texto 19/20...
Procesando texto 20/20...

Todos los resultados guardados en ..\resultados\06\a1_introduccion.json


In [5]:
import os
import json
import pandas as pd
from dotenv import load_dotenv
from openai import AzureOpenAI

load_dotenv()
API_KEY = os.getenv("AZURE_OPENAI_KEY")
ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
DEPLOYMENT_GPT = os.getenv("AZURE_OPENAI_DEPLOYMENT_GPT")
API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION")

client = AzureOpenAI(
    api_key=API_KEY,
    api_version=API_VERSION,
    azure_endpoint=ENDPOINT
)

data_path = os.path.join("..", "data", "escritos_seleccionados_a1.xlsx")
df = pd.read_excel(data_path)
textos = df["escrito"].dropna().tolist()

prompt_conclusion_cierre = """
Evalúa el siguiente texto usando esta rúbrica:

**Estructura global: Conclusión o cierre**
- Nivel 4: En la estructura del texto se distingue un cierre en el que se concluye o se proyecta la tesis.
- Nivel 3: En la estructura del texto se distingue un cierre a modo de comentario.
- Nivel 2: La estructura del texto no es clara, pero es posible inferir la presencia de una conclusión o cierre amalgamada en otras oraciones o párrafos.
- Nivel 1: No se identifica una conclusión o cierre en el texto.
Devuelve la evaluación **únicamente en formato JSON** con la siguiente estructura:

{{
  "conclusion_cierre": {{
    "nivel": (número de 1 a 4),
    "justificacion": "explicación breve"
}}

### Texto a evaluar:
{essay}

### INSTRUCCIONES CRÍTICAS:
- NO uses markdown, NO incluyas ```json, devuelve SOLO el JSON.
"""

resultados = []

for i, texto in enumerate(textos, start=1):    
    prompt_conclusion_cierre_a1 = prompt_conclusion_cierre.format(essay=texto)

    print(f"Procesando texto {i}/{len(textos)}...")

    messages = [
        {"role": "system", "content": 'Realiza las tareas de corrección que te solicite para un texto entregado'},
        {"role": "user", "content": prompt_conclusion_cierre_a1}
    ]

    try:
        response = client.chat.completions.create(
            model=DEPLOYMENT_GPT,
            messages=messages,
            # temperature=0, (No es soportado en Azure OpenAI)
            response_format={"type": "json_object"}
        )

        respuesta_limpia = response.choices[0].message.content.strip()
        resultado_json = json.loads(respuesta_limpia)
        resultados.append({
            "id_texto": i,
            "resultado": resultado_json
        })

    except json.JSONDecodeError as e:
        print(f"Error al parsear JSON en texto {i}: {e}")
        print("Respuesta cruda recibida:")
        print(respuesta_limpia)

output_dir = os.path.join("..", "resultados", "06")
os.makedirs(output_dir, exist_ok=True)
output_path = os.path.join(output_dir, "a1_conclusion_cierre.json")

with open(output_path, "w", encoding="utf-8") as f:
    json.dump(resultados, f, ensure_ascii=False, indent=4)

print(f"\nTodos los resultados guardados en {output_path}")

Procesando texto 1/20...
Procesando texto 2/20...
Procesando texto 3/20...
Procesando texto 4/20...
Procesando texto 5/20...
Procesando texto 6/20...
Procesando texto 7/20...
Procesando texto 8/20...
Procesando texto 9/20...
Procesando texto 10/20...
Procesando texto 11/20...
Procesando texto 12/20...
Procesando texto 13/20...
Procesando texto 14/20...
Procesando texto 15/20...
Procesando texto 16/20...
Procesando texto 17/20...
Procesando texto 18/20...
Procesando texto 19/20...
Procesando texto 20/20...

Todos los resultados guardados en ..\resultados\06\a1_conclusion_cierre.json


In [6]:
import os
import json
import pandas as pd
from dotenv import load_dotenv
from openai import AzureOpenAI

load_dotenv()
API_KEY = os.getenv("AZURE_OPENAI_KEY")
ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
DEPLOYMENT_GPT = os.getenv("AZURE_OPENAI_DEPLOYMENT_GPT")
API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION")

client = AzureOpenAI(
    api_key=API_KEY,
    api_version=API_VERSION,
    azure_endpoint=ENDPOINT
)

data_path = os.path.join("..", "data", "escritos_seleccionados_a1.xlsx")
df = pd.read_excel(data_path)
textos = df["escrito"].dropna().tolist()

prompt_progresion_textual = """
Evalúa el siguiente texto usando esta rúbrica:

**Progresión textual (uso de párrafos e inclusión de información principal)**
- Nivel 4: El texto presenta párrafos claramente identificables y en cada uno se reconoce una información principal distinta a la del párrafo anterior.
- Nivel 3: El texto presenta párrafos claramente identificables, pero en un párrafo se repite información de algún párrafo anterior.
- Nivel 2: El texto presenta párrafos claramente identificables, pero en dos párrafos se repite información de algún párrafo anterior.
- Nivel 1: No se presentan párrafos claramente identificables o en todos los párrafos se repite información anterior.

{{
  "progresion_textual": {{
    "nivel": (número de 1 a 4),
    "justificacion": "explicación breve"
}}

### Texto a evaluar:
{essay}

### INSTRUCCIONES CRÍTICAS:
- NO uses markdown, NO incluyas ```json, devuelve SOLO el JSON.
"""

resultados = []

for i, texto in enumerate(textos, start=1):    
    prompt_progresion_textual_a1 = prompt_progresion_textual.format(essay=texto)

    print(f"Procesando texto {i}/{len(textos)}...")

    messages = [
        {"role": "system", "content": 'Realiza las tareas de corrección que te solicite para un texto entregado'},
        {"role": "user", "content": prompt_progresion_textual_a1}
    ]

    try:
        response = client.chat.completions.create(
            model=DEPLOYMENT_GPT,
            messages=messages,
            # temperature=0, (No es soportado en Azure OpenAI)
            response_format={"type": "json_object"}
        )

        respuesta_limpia = response.choices[0].message.content.strip()
        resultado_json = json.loads(respuesta_limpia)
        resultados.append({
            "id_texto": i,
            "resultado": resultado_json
        })

    except json.JSONDecodeError as e:
        print(f"Error al parsear JSON en texto {i}: {e}")
        print("Respuesta cruda recibida:")
        print(respuesta_limpia)

output_dir = os.path.join("..", "resultados", "06")
os.makedirs(output_dir, exist_ok=True)
output_path = os.path.join(output_dir, "a1_progresion_textual.json")

with open(output_path, "w", encoding="utf-8") as f:
    json.dump(resultados, f, ensure_ascii=False, indent=4)

print(f"\nTodos los resultados guardados en {output_path}")

Procesando texto 1/20...
Procesando texto 2/20...
Procesando texto 3/20...
Procesando texto 4/20...
Procesando texto 5/20...
Procesando texto 6/20...
Procesando texto 7/20...
Procesando texto 8/20...
Procesando texto 9/20...
Procesando texto 10/20...
Procesando texto 11/20...
Procesando texto 12/20...
Procesando texto 13/20...
Procesando texto 14/20...
Procesando texto 15/20...
Procesando texto 16/20...
Procesando texto 17/20...
Procesando texto 18/20...
Procesando texto 19/20...
Procesando texto 20/20...

Todos los resultados guardados en ..\resultados\06\a1_progresion_textual.json
